# Descarga de datos fundamentales

## Cargamos Módulos

In [5]:
#Para quitar alarmas
import warnings
warnings.filterwarnings('ignore')
import yahoo_fin.stock_info as si
import yfinance as yf
import pandas as pd
from tqdm import tqdm

In [2]:
symbol = 'AAPL'
ticker = yf.Ticker(symbol)

### Estadisticas Básicas de las empresas
### 1. Get_quote_table
Tomamos GOOGLE como ejemplo

In [3]:
# Si da problemas dataframe append podemos bajar la version de pandas a la 1.4
# pip install pandas==1.4
quote = si.get_quote_table("aapl")

In [4]:
#veamos los valores
quote

{'1y Target Est': 199.57,
 '52 Week Range': '128.12 - 199.62',
 'Ask': '185.14 x 1100',
 'Avg. Volume': 54297359.0,
 'Beta (5Y Monthly)': 1.29,
 'Bid': '184.88 x 1000',
 "Day's Range": '182.73 - 185.15',
 'EPS (TTM)': 6.12,
 'Earnings Date': 'Jan 31, 2024 - Feb 05, 2024',
 'Ex-Dividend Date': 'Nov 10, 2023',
 'Forward Dividend & Yield': '0.96 (0.52%)',
 'Market Cap': '2.879T',
 'Open': 183.92,
 'PE Ratio (TTM)': 30.25,
 'Previous Close': 185.56,
 'Quote Price': 185.13999938964844,
 'Volume': 42377304.0}

### 2 Podemos obtener el valor de PE Ratio
A P/E ratio holds the most value to an analyst when compared against similar companies in the same industry or for a single company across a period of time.

In [4]:
quote["PE Ratio (TTM)"]

29.6

In [5]:
# Si convertimos el dict en un Dataframe
df_quote = pd.DataFrame.from_dict(quote, orient='index').rename(columns={0:'Valor'})

In [6]:
df_quote

,Valor
1y Target Est,199.57
52 Week Range,128.12 - 199.62
Ask,180.81 x 900
Avg. Volume,54028051.0
Beta (5Y Monthly),1.29
Bid,180.98 x 1200
Day's Range,180.17 - 182.76
EPS (TTM),6.12
Earnings Date,"Jan 31, 2024 - Feb 05, 2024"
Ex-Dividend Date,"Nov 10, 2023"


In [7]:
df_quote.iloc[df_quote.index.get_loc("PE Ratio (TTM)"),0]

29.6

#### 2.1 También con get_stats_valuation podemos obtener el trailing P/E

In [8]:
# También con get_stats_valuation podemos obtener el trailing P/E
val = si.get_stats_valuation("AAPL")

In [9]:
val

,0,1
0,Market Cap (intraday),2.82T
1,Enterprise Value,2.87T
2,Trailing P/E,29.56
3,Forward P/E,27.47
4,PEG Ratio (5 yr expected),2.17
5,Price/Sales (ttm),7.47
6,Price/Book (mrq),45.34
7,Enterprise Value/Revenue,7.48
8,Enterprise Value/EBITDA,22.20


In [12]:
val = val.iloc[:,:2]
val.columns = ["Attribute", "Recent"]

In [13]:
float(val[val.Attribute.str.contains("Trailing P/E")].iloc[0,1])

29.56

## 3 Cargamos todas las empresas del Índice Dow Jones y calculamos sus estadisticas fundamentales básicas

In [14]:
# get list of Dow tickers
dow_list = si.tickers_dow()

#Get data in the current column for each stock's valuation table
dow_stats = {}
for ticker in dow_list:
    temp = si.get_stats_valuation(ticker)
    temp = temp.iloc[:,:2]
    temp.columns = ["Attribute", "Recent"]
 
    dow_stats[ticker] = temp
# combine all the stats valuation tables into a single data frame
combined_stats = pd.concat(dow_stats)
combined_stats = combined_stats.reset_index()
 
del combined_stats["level_1"]
 
# update column names
combined_stats.columns = ["Ticker", "Attribute", "Recent"]

In [17]:
# Listamos Todos los parámetros de todas las empresas
combined_stats.head(12)

,Ticker,Attribute,Recent
0,AAPL,Market Cap (intraday),2.82T
1,AAPL,Enterprise Value,2.87T
2,AAPL,Trailing P/E,29.56
3,AAPL,Forward P/E,27.47
4,AAPL,PEG Ratio (5 yr expected),2.17
5,AAPL,Price/Sales (ttm),7.47
6,AAPL,Price/Book (mrq),45.34
7,AAPL,Enterprise Value/Revenue,7.48
8,AAPL,Enterprise Value/EBITDA,22.20
9,AMGN,Market Cap (intraday),162.16B


## 4 Conseguimos el ratio Price/Book para cada empresa

In [19]:
# Conseguimos el ratio Price/Book para cada empresa
combined_stats[combined_stats.Attribute.str.contains("Price/Book")]

,Ticker,Attribute,Recent
6,AAPL,Price/Book (mrq),45.34
15,AMGN,Price/Book (mrq),21.18
24,AXP,Price/Book (mrq),5.04
33,BA,Price/Book (mrq),NaN
42,CAT,Price/Book (mrq),7.18
51,CRM,Price/Book (mrq),4.18
60,CSCO,Price/Book (mrq),4.50
69,CVX,Price/Book (mrq),1.72
78,DIS,Price/Book (mrq),1.68
87,DOW,Price/Book (mrq),1.97


## 5 Obtenemos el ROE con get_stats
Return on equity (ROE) is a measure of financial performance calculated by dividing net income by shareholders' equity. Because shareholders' equity is equal to a company’s assets minus its debt, ROE is considered the return on net assets.

In [20]:
dow_extra_stats = {}
for ticker in tqdm(dow_list):
    dow_extra_stats[ticker] = si.get_stats(ticker)
    
combined_extra_stats = pd.concat(dow_extra_stats)
combined_extra_stats = combined_extra_stats.reset_index()
del combined_extra_stats["level_1"]
combined_extra_stats.columns = ["ticker", "Attribute", "Value"]

100%|██████████| 30/30 [00:36<00:00,  1.21s/it]


## 6 Descarga del ROE

In [21]:
# El ROE lo que hace es medir el beneficio de una compañía en relación con sus fondos propios para conocer la rentabilidad de la misma
combined_extra_stats[combined_extra_stats.Attribute.str.contains("Return on Equity")]

,ticker,Attribute,Value
34,AAPL,Return on Equity (ttm),171.95%
85,AMGN,Return on Equity (ttm),133.80%
136,AXP,Return on Equity (ttm),31.26%
187,BA,Return on Equity (ttm),NaN
238,CAT,Return on Equity (ttm),50.45%
289,CRM,Return on Equity (ttm),4.41%
340,CSCO,Return on Equity (ttm),31.78%
391,CVX,Return on Equity (ttm),15.68%
442,DIS,Return on Equity (ttm),3.06%
493,DOW,Return on Equity (ttm),7.24%


## How to get balance sheets with FMP API
La libreriá de Yahoo Fiannce no funciona correctamente vamos a hacerlo con el API de FMP 
Vamos a extraer los balances con esta API

#### Date de Alta en https://site.financialmodelingprep.com/ para obtener tu API KEY
#### Para instalar FMP API DSK
#pip install fmpsdk
#### El API Key debes guardarla en un fichero .env

In [30]:
import os
# Para cargar el fichero .env que debeis crear antes y salvarrlo en .env
from dotenv import load_dotenv
import typing
import fmpsdk

# Actual API key is stored in a .env file.  Not good to store API key directly in script.
load_dotenv()
apikey = os.environ.get("apikey")

# Company Valuation Methods
#symbol: str = "AAPL"
# print(f"Company Profile: {fmpsdk.company_profile(apikey=apikey, symbol=symbol)}")

## Listamos el total de capitalización de Apple de todos los balances

In [9]:
# Company Valuation Methods. Elegimos Apple
symbol: str = "AAPL"

## 7 Obtenemos los balances de los ultimos 4 años de Apple

In [88]:
# .balance_sheet_statement nos da una lista de diccionarios con los balances de los últimos 4 años
lstBalance = fmpsdk.balance_sheet_statement(apikey=apikey, symbol=symbol)

In [93]:
# lo convertimos a Datframe
dfBalance = pd.DataFrame(lstBalance)
dfBalance.columns

Index(['date', 'symbol', 'reportedCurrency', 'cik', 'fillingDate',
       'acceptedDate', 'calendarYear', 'period', 'cashAndCashEquivalents',
       'shortTermInvestments', 'cashAndShortTermInvestments', 'netReceivables',
       'inventory', 'otherCurrentAssets', 'totalCurrentAssets',
       'propertyPlantEquipmentNet', 'goodwill', 'intangibleAssets',
       'goodwillAndIntangibleAssets', 'longTermInvestments', 'taxAssets',
       'otherNonCurrentAssets', 'totalNonCurrentAssets', 'otherAssets',
       'totalAssets', 'accountPayables', 'shortTermDebt', 'taxPayables',
       'deferredRevenue', 'otherCurrentLiabilities', 'totalCurrentLiabilities',
       'longTermDebt', 'deferredRevenueNonCurrent',
       'deferredTaxLiabilitiesNonCurrent', 'otherNonCurrentLiabilities',
       'totalNonCurrentLiabilities', 'otherLiabilities',
       'capitalLeaseObligations', 'totalLiabilities', 'preferredStock',
       'commonStock', 'retainedEarnings',
       'accumulatedOtherComprehensiveIncomeLoss',
 

## 8 listamos Total Assets de los ultimos 4 años de Apple

In [92]:
dfBalance['totalAssets']

0    352583000000
1    352755000000
2    351002000000
3    323888000000
4    338516000000
Name: totalAssets, dtype: int64

# Dividendos - Estrategia “Dogs of Dow”
Vemos que 10 empresas del Dow Jones tienen mayores rentabilidad por dividendo

In [45]:
# get list of Dow tickers
dow_list = si.tickers_dow()

df = pd.DataFrame()
# creamos las columnas
df['DivYield'] = None
#Recorremos los tickers de Dow para conseguir sus precios y dividendos del ultimo año y calcular la rentabilidad por dividendos
dow_rentabilidad = {}

for ticker in dow_list:
    temp = pd.DataFrame(fmpsdk.historical_stock_dividend(apikey=apikey, symbol=symbol))
    dividendos = pd.concat([temp.drop(['historical'], axis=1), temp['historical'].apply(pd.Series)], axis=1)
    precio = pd.DataFrame(fmpsdk.company_profile(apikey=apikey, symbol=ticker))
    divYield = dividendos.iloc[0:4,3].sum()/precio['price']
    dow_rentabilidad[ticker] = divYield
    df.loc[ticker] = [divYield]

In [69]:
temp3 = pd.concat([df.drop(['DivYield'], axis=1), df['DivYield'].apply(pd.Series)], axis=1)
temp3 = temp3.rename({'0': 'RentaDiv'}, axis='columns')

In [68]:
temp3.sort_values(0, inplace=True, ascending=False)
temp3.head(10)

,0
WBA,0.037461
VZ,0.024334
INTC,0.019771
CSCO,0.019034
DOW,0.017612
KO,0.015833
DIS,0.010594
NKE,0.009247
MMM,0.008708
MRK,0.008022
